In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    token='')

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    token='')

#memory used 4544Mib

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
#task2

encoded = tokenizer.encode("What color is classic grass?", return_tensors="pt")
print("inputs id:")
print(encoded)

outputs = model.generate(
    encoded.to("cuda"),
    #max_length = 100,
    return_dict_in_generate=True,
    output_scores=True,
)

print()
print("output:")
for sentence in outputs[0]:
    print(tokenizer.decode(sentence))

#odp 1 - The tokenized input IDs are the unique identifiers of the input data. They are used to identify the input data in the system and to track its progress through the system.
#odp 2 - I’m trying to understand the model.generate() output, and I’m not sure what the keys are, and what they contain.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


inputs id:
tensor([[    1,  1824,  3181,   349, 11495, 10109, 28804]])


/home/student/miniconda3/envs/s207250/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(



output:
<s> What color is classic grass?

Green.

What color is the grass in the field?

Green.



In [ ]:
#task3

peftModel = PeftModel.from_pretrained(
    model,
    "alignment-handbook/zephyr-7b-sft-qlora", 
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    token='' #add token
)

peftTokenizer = AutoTokenizer.from_pretrained(
    "alignment-handbook/zephyr-7b-sft-qlora", 
    token='' #add token
)

In [8]:
#task 4

chatTemplateDict1 = [
    {"role": "system", "content": "you are villager in medival country"},
    {"role": "user", "content": "what is your favorite meal"},
    {"role": "assistant", "content": "my favorite meal is potato"}
]

peftTokenizer.apply_chat_template(chatTemplateDict1)

[523,
 28766,
 6574,
 28766,
 28767,
 13,
 7020,
 460,
 4314,
 1742,
 297,
 1342,
 3098,
 2939,
 2,
 28705,
 13,
 28789,
 28766,
 1838,
 28766,
 28767,
 13,
 6802,
 349,
 574,
 6656,
 11314,
 2,
 28705,
 13,
 28789,
 28766,
 489,
 11143,
 28766,
 28767,
 13,
 1916,
 6656,
 11314,
 349,
 2513,
 1827,
 2,
 28705,
 13]

In [9]:
peftTokenizer.chat_template

"{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

In [10]:
peftTokenizer.apply_chat_template(chatTemplateDict1, tokenize = False)

'<|system|>\nyou are villager in medival country</s>\n<|user|>\nwhat is your favorite meal</s>\n<|assistant|>\nmy favorite meal is potato</s>\n'

In [ ]:
chatTemplateDict2 = [
    {"role": "system", "content": "you are villager in medival country"},
    {"role": "system", "content": "your favorite meal is potato"},
    {"role": "system", "content": "you spend most of the days on your farmlands"},
    {"role": "system", "content": "you had 10 children but now you only have 3"}
]

peftTokenizer.apply_chat_template(chatTemplateDict2)